In [43]:
!pip install pandas


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [44]:
import pandas as pd
import os
from pathlib import Path

In [45]:
# Set up paths
data_folder = Path('/Users/navyaagarwal/Desktop/marketplace-ml-optimization/data')

# Get all CSV files
csv_files = sorted(data_folder.glob('*.csv'))

In [46]:
print("=" * 100)
print("DATA EXPLORATION REPORT")
print("=" * 100)
print(f"\nTotal CSV files found: {len(csv_files)}\n")

# Dictionary to store all dataset information
dataset_info = {}

for csv_file in csv_files:
    print(f"\n{'=' * 100}")
    print(f"FILE: {csv_file.name}")
    print(f"{'=' * 100}")

    # Read the CSV
    df = pd.read_csv(csv_file)

    # Store info
    dataset_info[csv_file.name] = {
        'shape': df.shape,
        'columns': df.columns.tolist(),
        'dtypes': df.dtypes.to_dict(),
        'file_size_mb': csv_file.stat().st_size / (1024 * 1024)
    }

    # Print basic info
    print(f"Shape (rows, columns): {df.shape}")
    print(f"File size: {csv_file.stat().st_size / (1024 * 1024):.2f} MB")
    print(f"\nColumns and Data Types:")
    print("-" * 80)
    for col, dtype in df.dtypes.items():
        print(f"  {col:<40} {str(dtype):<20}")

    # Print missing values
    print(f"\nMissing Values:")
    print("-" * 80)
    missing = df.isnull().sum()
    if missing.sum() > 0:
        for col, count in missing[missing > 0].items():
            percentage = (count / len(df)) * 100
            print(f"  {col:<40} {count:>10} ({percentage:>6.2f}%)")
    else:
        print("  No missing values found!")

    # Print basic statistics
    print(f"\nBasic Statistics:")
    print("-" * 80)
    print(f"  Total rows: {len(df):,}")
    print(f"  Total columns: {len(df.columns)}")
    print(f"  Memory usage: {df.memory_usage(deep=True).sum() / (1024 * 1024):.2f} MB")

    # Show first few rows
    print(f"\nFirst 5 rows:")
    print("-" * 80)
    print(df.head())

print("\n" + "=" * 100)
print("SUMMARY")
print("=" * 100)
print(f"\nTotal datasets: {len(dataset_info)}")
print(f"Total size: {sum(info['file_size_mb'] for info in dataset_info.values()):.2f} MB")
print(f"Total rows across all files: {sum(info['shape'][0] for info in dataset_info.values()):,}")

print("\nDataset Overview:")
print("-" * 80)
for filename, info in dataset_info.items():
    print(f"  {filename:<45} {info['shape'][0]:>10,} rows × {info['shape'][1]:>3} cols | {info['file_size_mb']:>7.2f} MB")


DATA EXPLORATION REPORT

Total CSV files found: 9


FILE: olist_customers_dataset.csv
Shape (rows, columns): (99441, 5)
File size: 8.62 MB

Columns and Data Types:
--------------------------------------------------------------------------------
  customer_id                              str                 
  customer_unique_id                       str                 
  customer_zip_code_prefix                 int64               
  customer_city                            str                 
  customer_state                           str                 

Missing Values:
--------------------------------------------------------------------------------
  No missing values found!

Basic Statistics:
--------------------------------------------------------------------------------
  Total rows: 99,441
  Total columns: 5
  Memory usage: 26.59 MB

First 5 rows:
--------------------------------------------------------------------------------
                        customer_id              

In [47]:
# Load all CSVs into dataframes with the same names as the files
olist_customers_dataset = pd.read_csv(data_folder / 'olist_customers_dataset.csv')
olist_geolocation_dataset = pd.read_csv(data_folder / 'olist_geolocation_dataset.csv')
olist_order_items_dataset = pd.read_csv(data_folder / 'olist_order_items_dataset.csv')
olist_order_payments_dataset = pd.read_csv(data_folder / 'olist_order_payments_dataset.csv')
olist_order_reviews_dataset = pd.read_csv(data_folder / 'olist_order_reviews_dataset.csv')
olist_orders_dataset = pd.read_csv(data_folder / 'olist_orders_dataset.csv')
olist_products_dataset = pd.read_csv(data_folder / 'olist_products_dataset.csv')
olist_sellers_dataset = pd.read_csv(data_folder / 'olist_sellers_dataset.csv')
product_category_name_translation = pd.read_csv(data_folder / 'product_category_name_translation.csv')

print("All datasets loaded successfully!")
print(f"Customers: {olist_customers_dataset.shape}")
print(f"Products: {olist_products_dataset.shape}")
print(f"Sellers: {olist_sellers_dataset.shape}")

All datasets loaded successfully!
Customers: (99441, 5)
Products: (32951, 9)
Sellers: (3095, 4)


In [48]:
# Merge products dataset with translation mapping
olist_products_dataset = olist_products_dataset.merge(
    product_category_name_translation,
    on='product_category_name',
    how='left'
)

# Replace the Portuguese column with English and drop the original Portuguese column
olist_products_dataset['product_category_name'] = olist_products_dataset['product_category_name_english']
olist_products_dataset = olist_products_dataset.drop(columns=['product_category_name_english'])

print("Product categories converted to English!")
print(olist_products_dataset[['product_id', 'product_category_name']].head(10))

Product categories converted to English!
                         product_id product_category_name
0  1e9e8ef04dbcff4541ed26657ea517e5             perfumery
1  3aa071139cb16b67ca9e5dea641aaa2f                   art
2  96bd76ec8810374ed1b65e291975717f        sports_leisure
3  cef67bcfe19066a932b7673e239eb23d                  baby
4  9dc1a7de274444849c219cff195d0b71            housewares
5  41d3672d4792049fa1779bb35283ed13   musical_instruments
6  732bd381ad09e530fe0a5f457d81becb            cool_stuff
7  2548af3e6e77a690cf3eb6368e9ab61e       furniture_decor
8  37cc742be07708b53a98702e77a21a02       home_appliances
9  8c92109888e8cdf9d66dc7e463025574                  toys


In [49]:
# For each zip code prefix, get the average lat/lon (in case there are multiple entries)
geo_coordinates = olist_geolocation_dataset.groupby('geolocation_zip_code_prefix')[
    ['geolocation_lat', 'geolocation_lng']
].mean().reset_index()

geo_coordinates.columns = ['zip_code_prefix', 'latitude', 'longitude']

# Add geolocation to customers
olist_customers_dataset = olist_customers_dataset.merge(
    geo_coordinates,
    left_on='customer_zip_code_prefix',
    right_on='zip_code_prefix',
    how='left'
)
olist_customers_dataset = olist_customers_dataset.drop(columns=['zip_code_prefix'])

# Add geolocation to sellers
olist_sellers_dataset = olist_sellers_dataset.merge(
    geo_coordinates,
    left_on='seller_zip_code_prefix',
    right_on='zip_code_prefix',
    how='left'
)
olist_sellers_dataset = olist_sellers_dataset.drop(columns=['zip_code_prefix'])

print("Geolocation coordinates added!")
print("\nCustomers with coordinates:")
print(olist_customers_dataset[['customer_id', 'customer_city', 'latitude', 'longitude']].head(5))
print(f"\nMissing lat/lon in customers: {olist_customers_dataset['latitude'].isna().sum()}")

print("\nSellers with coordinates:")
print(olist_sellers_dataset[['seller_id', 'seller_city', 'latitude', 'longitude']].head(5))
print(f"Missing lat/lon in sellers: {olist_sellers_dataset['latitude'].isna().sum()}")

Geolocation coordinates added!

Customers with coordinates:
                        customer_id          customer_city   latitude  \
0  06b8999e2fba1a1fbc88172c00ba8bc7                 franca -20.498489   
1  18955e83d337fd6b2def6b18a428ac77  sao bernardo do campo -23.727992   
2  4e7b3e00288586ebd08712fdd0374a03              sao paulo -23.531642   
3  b2b6027bc5c5109e529d4dc6358b12c3        mogi das cruzes -23.499702   
4  4f2d8ab171c80ec8364f7c12e35b23ad               campinas -22.975100   

   longitude  
0 -47.396929  
1 -46.542848  
2 -46.656289  
3 -46.185233  
4 -47.142925  

Missing lat/lon in customers: 278

Sellers with coordinates:
                          seller_id        seller_city   latitude  longitude
0  3442f8959a84dea7ee197c632cb2df15           campinas -22.893848 -47.061337
1  d1b65fc7debc3361ea86b5f14c68d2e2         mogi guacu -22.383437 -46.947927
2  ce3ad9de960102d0677a81f5d0bb7b2d     rio de janeiro -22.909572 -43.177703
3  c0f3eea2e14555b6faeea3dd58c1b1c3      

In [50]:
# Find customers with missing lat/lon
customers_missing = olist_customers_dataset[olist_customers_dataset['latitude'].isna()]

print("CUSTOMERS WITH MISSING LAT/LON")
print("=" * 80)
print(f"Count: {len(customers_missing)}\n")

# Show which zip code prefixes are missing
missing_zip_codes = customers_missing['customer_zip_code_prefix'].unique()
print(f"Unique zip code prefixes with missing coordinates: {missing_zip_codes}\n")

# Check if these zip codes exist in the geolocation dataset
print("Checking if these zip codes exist in geolocation dataset:")
for zip_code in missing_zip_codes:
    exists = zip_code in olist_geolocation_dataset['geolocation_zip_code_prefix'].values
    print(f"  Zip {zip_code}: {'EXISTS' if exists else 'NOT FOUND'}")

# Show sample of customers with missing coordinates
print("\nSample of customers with missing coordinates:")
print(customers_missing[['customer_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'latitude']].head(10))

CUSTOMERS WITH MISSING LAT/LON
Count: 278

Unique zip code prefixes with missing coordinates: [72300 11547 64605 72465  7729 72904 35408 78554 73369  8980 29949 65137
 28655 73255 28388  6930 71676 64047 61906 83210 71919 36956 35242 72005
 29718 41347 70324 70686 72341 12332 70716 71905 75784 73082 71884 71574
 72238 71996 76968 71975 72595 72017 72596 67105 25840 72002 72821 85118
 25919 95853 72583 68511 70701 71591 72535 95572 73090 72242 86135 70316
 73091 41098 58734 73310 71810 72280  7430 73081 70333 72268 35104 72455
 72237 17390 76897 84623 70702 72760 73088 29196 36596 57254 71995 73093
 75257 48504 83843 62625 37005 73401 49870 13307 28617 73402 56327 71976
 72587 85958 19740 77404 44135 28120 72863 87323 87511 72440 72243 65830
 71261 28575  2140 71551 72023 28160 55027 43870 94370 38710 42716 36248
 71593 71953 72549 72457 56485 71590 93602  7412  8342 39103 72536 59547
 59299 85894 36857 71993  7784 71539 42843 86996 73272 72867 27980 58286
 71208 71971 73391 64095 72427

In [51]:
# Find sellers with missing lat/lon
sellers_missing = olist_sellers_dataset[olist_sellers_dataset['latitude'].isna()]

print("\nSELLERS WITH MISSING LAT/LON")
print("=" * 80)
print(f"Count: {len(sellers_missing)}\n")

# Show which zip code prefixes are missing
missing_zip_codes = sellers_missing['seller_zip_code_prefix'].unique()
print(f"Unique zip code prefixes with missing coordinates: {missing_zip_codes}\n")

# Check if these zip codes exist in the geolocation dataset
print("Checking if these zip codes exist in geolocation dataset:")
for zip_code in missing_zip_codes:
    exists = zip_code in olist_geolocation_dataset['geolocation_zip_code_prefix'].values
    print(f"  Zip {zip_code}: {'EXISTS' if exists else 'NOT FOUND'}")

# Show sample of sellers with missing coordinates
print("\nSample of sellers with missing coordinates:")
print(sellers_missing[['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state', 'latitude']])


SELLERS WITH MISSING LAT/LON
Count: 7

Unique zip code prefixes with missing coordinates: [82040 91901 72580  2285  7412 71551 37708]

Checking if these zip codes exist in geolocation dataset:
  Zip 82040: NOT FOUND
  Zip 91901: NOT FOUND
  Zip 72580: NOT FOUND
  Zip 2285: NOT FOUND
  Zip 7412: NOT FOUND
  Zip 71551: NOT FOUND
  Zip 37708: NOT FOUND

Sample of sellers with missing coordinates:
                             seller_id  seller_zip_code_prefix  \
473   5962468f885ea01a1b6a97a218797b0a                   82040   
791   2aafae69bf4c41fbd94053d9413e87ee                   91901   
1672  2a50b7ee5aebecc6fd0ff9784a4747d6                   72580   
1931  2e90cb1677d35cfe24eef47d441b7c87                    2285   
2182  0b3f27369a4d8df98f7eb91077e438ac                    7412   
2986  42bde9fef835393bb8a8849cb6b7f245                   71551   
3028  870d0118f7a9d85960f29ad89d5d989a                   37708   

          seller_city seller_state  latitude  
473          curitiba     

In [52]:
print("BEFORE DROPPING RECORDS")
print("=" * 80)
print(f"Customers: {len(olist_customers_dataset)} rows")
print(f"Sellers: {len(olist_sellers_dataset)} rows")

# Drop customers with missing lat/lon
olist_customers_dataset = olist_customers_dataset.dropna(subset=['latitude', 'longitude'])

# Drop sellers with missing lat/lon
olist_sellers_dataset = olist_sellers_dataset.dropna(subset=['latitude', 'longitude'])

print("\nAFTER DROPPING RECORDS")
print("=" * 80)
print(f"Customers: {len(olist_customers_dataset)} rows (dropped {99441 - len(olist_customers_dataset)})")
print(f"Sellers: {len(olist_sellers_dataset)} rows (dropped {3095 - len(olist_sellers_dataset)})")

print("\nVerification - any remaining missing coordinates?")
print(f"Customers missing lat/lon: {olist_customers_dataset['latitude'].isna().sum()}")
print(f"Sellers missing lat/lon: {olist_sellers_dataset['latitude'].isna().sum()}")

BEFORE DROPPING RECORDS
Customers: 99441 rows
Sellers: 3095 rows

AFTER DROPPING RECORDS
Customers: 99163 rows (dropped 278)
Sellers: 3088 rows (dropped 7)

Verification - any remaining missing coordinates?
Customers missing lat/lon: 0
Sellers missing lat/lon: 0


In [53]:
olist_customers_dataset = olist_customers_dataset.drop(
    columns=[
        "customer_city",
        "customer_state",
        "customer_zip_code_prefix"
    ]
)

In [54]:
olist_sellers_dataset = olist_sellers_dataset.drop(
    columns=[
        "seller_city",
        "seller_state",
        "seller_zip_code_prefix"
    ]
)

In [55]:
# FINAL DATAFRAMES TO ANALYZE

print(olist_customers_dataset.head())
print("======================================================================================")
print(olist_order_items_dataset.head())
print("======================================================================================")
print(olist_order_payments_dataset.head())
print("======================================================================================")
print(olist_order_reviews_dataset.head())
print("======================================================================================")
print(olist_orders_dataset.head())
print("======================================================================================")
print(olist_products_dataset.head())
print("======================================================================================")
print(olist_sellers_dataset.head())
print("======================================================================================")

                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

    latitude  longitude  
0 -20.498489 -47.396929  
1 -23.727992 -46.542848  
2 -23.531642 -46.656289  
3 -23.499702 -46.185233  
4 -22.975100 -47.142925  
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244

In [56]:
olist_orders_dataset.info()

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   order_id                       99441 non-null  str  
 1   customer_id                    99441 non-null  str  
 2   order_status                   99441 non-null  str  
 3   order_purchase_timestamp       99441 non-null  str  
 4   order_approved_at              99281 non-null  str  
 5   order_delivered_carrier_date   97658 non-null  str  
 6   order_delivered_customer_date  96476 non-null  str  
 7   order_estimated_delivery_date  99441 non-null  str  
dtypes: str(8)
memory usage: 6.1 MB


In [57]:
dataframes = [
    ("customers", olist_customers_dataset),
    ("order_items", olist_order_items_dataset),
    ("order_payments", olist_order_payments_dataset),
    ("order_reviews", olist_order_reviews_dataset),
    ("orders", olist_orders_dataset),
    ("products", olist_products_dataset),
    ("sellers", olist_sellers_dataset),
]

In [58]:
for name, df in dataframes:
    print(f"\n================ {name.upper()} =================")
    print("Shape:", df.shape)
    print("\nData Types:")
    print(df.dtypes)
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nUnique Values Per Column:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()}")


================ CUSTOMERS =================
Shape: (99163, 4)

Data Types:
customer_id               str
customer_unique_id        str
latitude              float64
longitude             float64
dtype: object

Missing Values:
customer_id           0
customer_unique_id    0
latitude              0
longitude             0
dtype: int64

Unique Values Per Column:
customer_id: 99163
customer_unique_id: 95828
latitude: 14837
longitude: 14837

================ ORDER_ITEMS =================
Shape: (112650, 7)

Data Types:
order_id                   str
order_item_id            int64
product_id                 str
seller_id                  str
shipping_limit_date        str
price                  float64
freight_value          float64
dtype: object

Missing Values:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

Unique Values Per Column:
order_id: 98666

In [59]:
olist_order_items_dataset["shipping_limit_date"] = pd.to_datetime(
    olist_order_items_dataset["shipping_limit_date"]
)

In [60]:
date_cols_reviews = [
    "review_creation_date",
    "review_answer_timestamp"
]

for col in date_cols_reviews:
    olist_order_reviews_dataset[col] = pd.to_datetime(
        olist_order_reviews_dataset[col]
    )

In [61]:
date_cols_orders = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in date_cols_orders:
    olist_orders_dataset[col] = pd.to_datetime(
        olist_orders_dataset[col]
    )

In [62]:
olist_orders_dataset.groupby("order_status").size()

order_status
approved           2
canceled         625
created            5
delivered      96478
invoiced         314
processing       301
shipped         1107
unavailable      609
dtype: int64

In [63]:
olist_products_dataset = olist_products_dataset.dropna(
    subset=["product_photos_qty", "product_description_lenght"]
)

In [64]:
# FINAL DATAFRAMES TO ANALYZE

print(olist_customers_dataset.head())
print("======================================================================================")
print(olist_order_items_dataset.head())
print("======================================================================================")
print(olist_order_payments_dataset.head())
print("======================================================================================")
print(olist_order_reviews_dataset.head())
print("======================================================================================")
print(olist_orders_dataset.head())
print("======================================================================================")
print(olist_products_dataset.head())
print("======================================================================================")
print(olist_sellers_dataset.head())
print("======================================================================================")

                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

    latitude  longitude  
0 -20.498489 -47.396929  
1 -23.727992 -46.542848  
2 -23.531642 -46.656289  
3 -23.499702 -46.185233  
4 -22.975100 -47.142925  
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id  \
0  4244

In [65]:
for name, df in dataframes:
    print(f"\n================ {name.upper()} =================")
    print("Shape:", df.shape)
    print("\nData Types:")
    print(df.dtypes)
    print("\nMissing Values:")
    print(df.isnull().sum())
    print("\nUnique Values Per Column:")
    for col in df.columns:
        print(f"{col}: {df[col].nunique()}")


================ CUSTOMERS =================
Shape: (99163, 4)

Data Types:
customer_id               str
customer_unique_id        str
latitude              float64
longitude             float64
dtype: object

Missing Values:
customer_id           0
customer_unique_id    0
latitude              0
longitude             0
dtype: int64

Unique Values Per Column:
customer_id: 99163
customer_unique_id: 95828
latitude: 14837
longitude: 14837

================ ORDER_ITEMS =================
Shape: (112650, 7)

Data Types:
order_id                          str
order_item_id                   int64
product_id                        str
seller_id                         str
shipping_limit_date    datetime64[us]
price                         float64
freight_value                 float64
dtype: object

Missing Values:
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype:

In [67]:
order_items_agg = (
    olist_order_items_dataset
    .groupby("order_id")
    .agg(
        total_price=("price", "sum"),
        total_freight=("freight_value", "sum")
    )
    .reset_index()
)

order_items_agg["items_total"] = (
    order_items_agg["total_price"] +
    order_items_agg["total_freight"]
)

In [68]:
payments_agg = (
    olist_order_payments_dataset
    .groupby("order_id")
    .agg(
        total_payment=("payment_value", "sum")
    )
    .reset_index()
)

In [69]:
payment_comparison = (
    order_items_agg
    .merge(payments_agg, on="order_id", how="inner")
)

In [70]:
payment_comparison["difference"] = (
    payment_comparison["total_payment"] -
    payment_comparison["items_total"]
)

In [71]:
payment_comparison["difference"].describe()

count    98665.000000
mean         0.029092
std          1.129221
min        -51.620000
25%          0.000000
50%          0.000000
75%          0.000000
max        182.810000
Name: difference, dtype: float64

In [72]:
payment_with_installments = (
    payment_comparison
    .merge(
        olist_order_payments_dataset[["order_id", "payment_installments"]],
        on="order_id",
        how="left"
    )
)

payment_with_installments.groupby("payment_installments")["difference"].mean()

payment_installments
0     0.000000e+00
1    -8.788380e-04
2     1.268941e-03
3     9.453042e-03
4     1.877872e-02
5     2.512885e-02
6     6.711721e-02
7     1.120062e-01
8     5.648566e-02
9     1.706299e-01
10    1.996454e-01
11    3.116364e+00
12    2.653588e+00
13    1.570000e+00
14   -3.789561e-15
15    8.998649e-01
16   -5.684342e-15
17   -8.120488e-15
18   -6.315935e-15
20    6.581765e+00
21    2.033667e+01
22    0.000000e+00
23    0.000000e+00
24    3.426667e+00
Name: difference, dtype: float64

In [76]:
total_extra_revenue = payment_comparison["difference"].sum()
print(total_extra_revenue)

2870.389999999961


In [78]:
interest_revenue = payment_comparison["difference"].sum()
total_revenue = payment_comparison["total_payment"].sum()

interest_share = interest_revenue / total_revenue
print(interest_share)

0.00018113967247872792


In [79]:
olist_order_payments_dataset["payment_type"].value_counts()

payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64

In [80]:
payment_detailed = (
    payment_comparison
    .merge(
        olist_order_payments_dataset[
            ["order_id", "payment_type", "payment_installments"]
        ],
        on="order_id",
        how="left"
    )
)

In [81]:
credit_card_payments = payment_detailed[
    payment_detailed["payment_type"] == "credit_card"
]
installment_payments = credit_card_payments[
    credit_card_payments["payment_installments"] > 1
]

inst_total_revenue = installment_payments["total_payment"].sum()
inst_interest = installment_payments["difference"].sum()

inst_interest_share = inst_interest / inst_total_revenue
print(inst_interest_share)

0.0002917966989086522


In [82]:
order_items_agg = (
    olist_order_items_dataset
    .groupby("order_id")
    .agg(
        total_price=("price", "sum"),
        total_freight=("freight_value", "sum"),
        total_items=("order_item_id", "count")
    )
    .reset_index()
)

order_items_agg["items_total"] = (
    order_items_agg["total_price"] +
    order_items_agg["total_freight"]
)

In [84]:
payments_agg = (
    olist_order_payments_dataset
    .groupby("order_id")
    .agg(
        total_payment=("payment_value", "sum"),
        max_installments=("payment_installments", "max")
    )
    .reset_index()
)
reviews_agg = (
    olist_order_reviews_dataset
    .groupby("order_id")
    .agg(
        review_score=("review_score", "mean")
    )
    .reset_index()
)


In [85]:
orders_master = (
    olist_orders_dataset
    .merge(order_items_agg, on="order_id", how="left")
    .merge(payments_agg, on="order_id", how="left")
    .merge(reviews_agg, on="order_id", how="left")
)

In [86]:
orders_master["revenue_diff"] = (
    orders_master["total_payment"] -
    orders_master["items_total"]
)

In [88]:
negative_orders = orders_master[
    orders_master["revenue_diff"] < 0
]
print("Negative Orders:", len(negative_orders))
print("Total Negative Revenue:", negative_orders["revenue_diff"].sum())
len(negative_orders) / len(orders_master)

Negative Orders: 9716
Total Negative Revenue: -200.78000000022638


0.09770617753240615

In [89]:
orders_master["material_negative_flag"] = (
    orders_master["revenue_diff"] < -5
).astype(int)
material_negatives = orders_master[
    orders_master["revenue_diff"] < -5
]

print(len(material_negatives))
print(material_negatives["revenue_diff"].sum())

11
-179.07999999999998


In [90]:
orders_master["revenue_diff"].quantile([0.01, 0.05, 0.95, 0.99])

0.01   -5.684342e-14
0.05   -1.421085e-14
0.95    1.421085e-14
0.99    5.684342e-14
Name: revenue_diff, dtype: float64

In [91]:
orders_master["freight_ratio"] = (
    orders_master["total_freight"] /
    orders_master["total_price"]
)

orders_master["freight_ratio"].describe()
orders_master["freight_ratio"].quantile([0.5, 0.75, 0.9, 0.95, 0.99])

0.50    0.224374
0.75    0.380191
0.90    0.622503
0.95    0.839355
0.99    1.461909
Name: freight_ratio, dtype: float64

In [94]:
# Create path
eda_path = os.path.join(data_folder, "clean_data")
os.makedirs(eda_path, exist_ok=True)

# Save datasets
orders_master.to_csv(os.path.join(eda_path, "orders_master.csv"), index=False)

olist_customers_dataset.to_csv(os.path.join(eda_path, "customers.csv"), index=False)
olist_order_items_dataset.to_csv(os.path.join(eda_path, "order_items.csv"), index=False)
olist_order_payments_dataset.to_csv(os.path.join(eda_path, "order_payments.csv"), index=False)
olist_order_reviews_dataset.to_csv(os.path.join(eda_path, "order_reviews.csv"), index=False)
olist_orders_dataset.to_csv(os.path.join(eda_path, "orders.csv"), index=False)
olist_products_dataset.to_csv(os.path.join(eda_path, "products.csv"), index=False)
olist_sellers_dataset.to_csv(os.path.join(eda_path, "sellers.csv"), index=False)